In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier


In [6]:
# 🧼 Re-load clean version of train & test
train = pd.read_csv('Train_Data.csv')
test = pd.read_csv('Test_Data.csv')


In [7]:
print(train.columns)

Index(['SEQN', 'RIAGENDR', 'PAQ605', 'BMXBMI', 'LBXGLU', 'DIQ010', 'LBXGLT',
       'LBXIN', 'age_group'],
      dtype='object')


In [8]:
# 2. Extract and fix y
y = train['age_group']

# Convert strings to numbers
y = y.replace({'Adult': 0, 'Senior': 1})

# 🩹 Handle missing target values
y = y.dropna()

# Convert to int safely
y = y.astype(int)

# 3. Align X with filtered y
train = train.loc[y.index]  # Keep only rows where y exists

# 4. Drop unused columns
train.drop(columns=['SEQN', 'age_group'], inplace=True)
test_ids = test['SEQN']
test.drop(columns=['SEQN'], inplace=True)

C:\Users\Arnav Mathur\AppData\Local\Temp\ipykernel_20396\1181082223.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'Adult': 0, 'Senior': 1})


In [9]:
features_to_impute = ['BMXBMI', 'LBXGLU', 'LBXGLT', 'LBXIN']

for col in features_to_impute:
    train[col] = train[col].fillna(train.groupby('RIAGENDR')[col].transform('median'))
    test[col] = test[col].fillna(train.groupby('RIAGENDR')[col].transform('median'))


In [10]:
def feature_engineering(df):
    df['GLU_INS_Ratio'] = df['LBXGLU'] / (df['LBXIN'] + 1e-3)
    df['BMI_Obese'] = (df['BMXBMI'] > 30).astype(int)
    df['BMI_Underweight'] = (df['BMXBMI'] < 18.5).astype(int)
    df['High_Glucose'] = (df['LBXGLU'] > 125).astype(int)
    df['Low_Glucose'] = (df['LBXGLU'] < 70).astype(int)
    df['Insulin_High'] = (df['LBXIN'] > 25).astype(int)
    return df

train = feature_engineering(train)
test = feature_engineering(test)


In [11]:
combined = pd.concat([train, test], axis=0)
combined = pd.get_dummies(combined, columns=['RIAGENDR', 'DIQ010'], drop_first=True)

X = combined.iloc[:len(train)].copy()
X_test = combined.iloc[len(train):].copy()

X = combined.iloc[:len(train)].copy()

model = XGBClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model.fit(X, y.values.ravel())



c:\Users\Arnav Mathur\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:27:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, ...)

In [12]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
print("Cross-validation Accuracy:", np.round(scores.mean(), 4))


c:\Users\Arnav Mathur\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:27:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Arnav Mathur\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:27:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Arnav Mathur\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:27:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Arnav Mathur\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: 

Cross-validation Accuracy: 0.8315


c:\Users\Arnav Mathur\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:27:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [13]:
predictions = model.predict(X_test)

In [14]:
submission = pd.DataFrame({'age_group': predictions})
submission.to_csv('submission.csv', index=False)
print("✅ Submission file saved as 'submission.csv'")


✅ Submission file saved as 'submission.csv'
